In [28]:
# 네이버 검색 API예제는 블로그를 비롯 전문자료까지 호출방법이 동일하므로 blog검색만 대표로 예제를 올렸습니다.
# 네이버 검색 Open API 예제 - 블로그 검색
import os
import sys
import urllib
import urllib3
import requests
import json
import xmltodict
import re
from bs4 import BeautifulSoup
from collections import defaultdict
import pandas as pd
import numpy as np
from typing import Union
from tqdm import tqdm
from lxml import etree
from copy import deepcopy
import pickle

# Hide Warnings
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

client_id = "VyJ4vDh18O7CVkYDCROr"
client_secret = "pOn2vw75sv"

# 프라엘 갈바닉, LG전자 디오스 냉장고, 스피커, LG홈브루
# SEARCH_KEYWORD = "프라엘 갈바닉"
# SEARCH_KEYWORDS = ["스메그냉장고", "다이슨선풍기", "LG프라엘", "힐링미안마의자", "스탠바이미", "발뮤다토스터","다이슨에어랩",
#                 "LG공기청정기", "LG퓨리케어공기청정기", "LG스타일러", "단미와플메이커",
#                "오아가습기", "LG시네빔", "LG올레드tv", "lgoledtv", "뱅앤올룹슨", "bangandolufsen", "프라엘갈바닉", 
#                "LG디오스냉장고", "디오스냉장고", "마샬스피커", "LG홈브루"]
               
# SEARCH_TAGS = ["스메그냉장고", "마샬스피커", "피닉스마사지건", "LG디오스냉장고", "LG냉장고", "다이슨청소기", "드롱기커피머신", "드롱기토스트기", "한샘칫솔살균기",
#                "디디오랩에어프라이어", "로라스타다리미", "벨레스피커", "브레오두피마사지기", "벤타에어워셔", "보이로전기요", "비쎌스팀청소기", "필립스에어프라이어",
#                "일리커피머신", "제네바스피커", "크로슬리턴테이블", "티악턴테이블", "파세코창문형에어컨", "파나소닉안마의자", "스메그오븐", "크로슬리턴테이블",
#                "스메그토스트기", "스메그커피머신", "벤타에어워셔", "모슈텀블러", "네스프레소버츄오", "다이슨슈퍼소닉", "일월온수매트", "제니퍼룸가습기", "하만카돈사운드바"]

# # url = "https://openapi.naver.com/v1/search/blog.xml?query=" + encText # xml 결과


In [36]:
product_master = pd.read_excel("../Naver_Shopping/상품_정보_마스터_220311.xlsx")

In [ ]:
product_master

In [129]:
target_product_master = product_master.loc[product_master[" 상품 대상여부"].isin(["O", "△"])]
SEARCH_KEYWORDS = target_product_master["상품명"].to_list()

In [57]:
with open("search_keywords.pkl", "wb") as f:
    pickle.dump(search_keywords, f)

In [56]:
search_keywords = []
for iterrow in product_master.iterrows():
    row = iterrow[1]

    if row['브랜드'] is not np.nan:
        search_keywords.append(f"{row['브랜드']} {row['상품명']}")
    elif row['제조사'] is not np.nan:
        search_keywords.append(f"{row['제조사']} {row['상품명']}")
    else:
        search_keywords.append(row["상품명"])

    # else:
    #     if row['제조사'] is not np.nan:
    #         manufacture_category.append(f"{row['제조사']} {row['소 카테고리']}")
    #     if row['브랜드'] is not np.nan:            
    #         brand_category.append(f"{row['브랜드']} {row['소 카테고리']}")


In [48]:
SEARCH_KEYWORDS = np.unique(brand_category + manufacture_category).tolist()
len(SEARCH_KEYWORDS)

1105

In [9]:
blog_soup = BeautifulSoup(requests.get(url, verify = False).text, "html.parser")

In [12]:
blog_soup.__format__

<function BeautifulSoup.__format__(format_spec, /)>

In [11]:
isinstance(blog_soup, 

False

In [4]:
url = "https://m.blog.naver.com/PostView.naver?blogId=herdsman77&logNo=222323799763&proxyReferer="
print(naver_blog_crawl.blog_url)
blog_soup = BeautifulSoup(requests.get(naver_blog_crawl.blog_url, verify = False).text, "html.parser")

NameError: name 'naver_blog_crawl' is not defined

In [23]:
blog_soup


<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">

<html lang="ko">
<head>
<meta content="no-cache" http-equiv="Pragma"/>
<meta content="-1" http-equiv="Expires"/>
<meta content="always" name="referrer"/>
<!--[if ie]>
<style type="text/css">
html {overflow: scroll; overflow-x: auto;}
</style>
<![endif]-->
<link charset="UTF-8" href="https://ssl.pstatic.net/t.static.blog/mylog/versioning/LayoutTopCommon-551149225_https.css" rel="stylesheet" type="text/css"/>
<link href="https://ssl.pstatic.net/t.static.blog/mylog/versioning//common/css/music/player-d3fc09e_https.css" rel="stylesheet" type="text/css"/>
<link href="/favicon.ico?3" rel="shortcut icon" type="image/x-icon">
<meta content="한샘 칫솔살균기 써보고 반했어요" property="og:title"/>
<meta content="https://blogthumb.pstatic.net/MjAyMTAxMDZfMjAw/MDAxNjA5OTQyNTk4Mjcy.9WrDGlN7GijZU6cHpYvEgqP7r1dq2ZszksCvgzRvoeAg.-Tor7UZpjOSU4fQDLV20sCQxdmc12YrAFbG5DWwWgdwg.JPEG.hismoky0408/13.jpg?type=w2" propert

In [10]:
blog_contents_dict = defaultdict(list)

# 본문 내에 Image와 Text가 있는 부분만 추출
blog_body = blog_soup.find("div", attrs = {"id":"postViewArea"})
blog_img_txt = blog_body.findAll("p")


In [16]:
blog_img_txt[0].find("img", attrs={"class": "_photoImage"})

In [ ]:

# image 파일 이름과 블로그 글을 하나의 contents로 엮어서 text 생성
raw_blog_contents = []
img_urls = []
img_names = []
img_num = 0
for x in blog_img_txt:
    if x.find("img", attrs={"class": "_photoImage"}) is not None:
        img_num += 1
        if "data-lazy-src" in x.select_one("img", attrs={"class": "_photoImage"}).attrs.keys():
            img_urls.append(x.select_one("img", attrs={"class": "_photoImage"})["data-lazy-src"])
        else:
            img_urls.append(x.select_one("img", attrs={"class": "_photoImage"})["src"])
        bloggername = re.sub(r'/|\.|\*|%|\\|:|\?|\"|\'|\<|\>|\|','_',bloggername)
        img_name = f"{self.keyword}_{bloggername}_{postdate}_img_{img_num}.jpg"
        img_names.append(img_name)
        raw_blog_contents.append(img_name)

        text = x.get_text()
        raw_blog_contents.append(text)
    else:
        text = x.get_text()
        raw_blog_contents.append(text)

blog_contents = "\n".join(raw_blog_contents)

blog_contents_dict["images"] = img_names
blog_contents_dict["image_urls"] = img_urls
blog_contents_dict["contents"] = blog_contents


In [24]:
blog_contents_dict = defaultdict(list)

blog_body = blog_soup.find("div", attrs={"class":"se-main-container"})

# 본문 내에 Image와 Text가 있는 부분만 추출
blog_img_txt = blog_body.findAll("div", attrs={"class" : [re.compile(r"se-(module|section)-text"), re.compile(r"se-(module|section)-image")]})


In [ ]:
blog_body.findAll("div", attrs={"class" : [re.compile(r"se-(module|section)-text"), re.compile(r"se-(module|section)-image")]})

In [38]:
blog_img_txt[0].select_one("img")

<img alt="" class="se-image-resource" data-height="590" data-lazy-src="https://mblogthumb-phinf.pstatic.net/MjAyMTA0MjdfMTkz/MDAxNjE5NTE0MTMzMTY0.m9M-AflEDCR8EN6PN6EuRwxovKlVZarCvnFpWUYZsf8g.0-iCeduazKxNDC5L2N6xjqWJTeygZzl_TSK6Qhb2LTIg.JPEG.herdsman77/00.jpg?type=w800" data-width="886" src="https://mblogthumb-phinf.pstatic.net/MjAyMTA0MjdfMTkz/MDAxNjE5NTE0MTMzMTY0.m9M-AflEDCR8EN6PN6EuRwxovKlVZarCvnFpWUYZsf8g.0-iCeduazKxNDC5L2N6xjqWJTeygZzl_TSK6Qhb2LTIg.JPEG.herdsman77/00.jpg?type=w80_blur">
</img>

In [29]:
blog_img_txt[31].select_one_)

<div class="se-section se-section-image se-l-default se-section-align-">
<div class="se-module se-module-image se-module-image-360vr">
<div class="se-360vr-preview" style="background-image:url('https://postfiles.pstatic.net/MjAyMTAxMDZfMTY5/MDAxNjA5OTQzOTI2NTQz.RRNQHi7PkPpvLXFw35H1I9CKbr_mxi8iofutRZ-VV5Qg.W8o9EPPS6V2X5_NiJEK5BdAvAxV_KLGOLA3WYemn-cYg.JPEG.hismoky0408/PM-1.P-pa/29.jpg?type=preview966_544');"></div>
</div> </div>

In [28]:
# image 파일 이름과 블로그 글을 하나의 contents로 엮어서 text 생성
raw_blog_contents = []
img_urls = []
img_names = []
img_num = 0
for i, x in enumerate(blog_img_txt):
    print(i)
    if any([re.findall(r"se-module-image|se-section-image", y) for y in x.attrs["class"]]):
        
        if "data-lazy-src" in x.select_one("img").attrs.keys():
            img_url = x.select_one("img")["data-lazy-src"]
        else:
            img_url = x.select_one("img")["src"]
        
        if img_url in img_urls:
            continue
        else:
            img_urls.append(img_url)

        # bloggername = re.sub(r'/|\.|\*|%|\\|:|\?|\"|\'|\<|\>|\|','_',bloggername)
        
        img_num += 1
        img_name = f"img_{img_num}.jpg"
        img_names.append(img_name)
        raw_blog_contents.append(img_name)
        
    elif "se-module-text" in x.attrs["class"]:
        text = x.get_text()
        raw_blog_contents.append(text)

blog_contents = "\n".join(raw_blog_contents)

blog_contents_dict["contents"] = blog_contents
blog_contents_dict["images"] = img_names
blog_contents_dict["image_urls"] = img_urls

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31


AttributeError: 'NoneType' object has no attribute 'attrs'

In [35]:
naver_blog_crawl.blog_infos["bloggernames"]

['둥지니월드',
 None,
 'Blueholic_',
 '단간우',
 'GATEVISION 다이슨 로라스타 이메텍 비쎌 노바이러스',
 '레몬이의 life',
 '예시카의 이층집',
 '마코’s 매일 성장 프로젝트',
 '★포푸리 착한리뷰',
 '오리의 퍼스널 뷰티',
 '무크의 리뷰일상',
 '슬기로운 엄마생활',
 '찹쌀에디션',
 '함영민의 디카갤러리',
 '둘째공주',
 '트레블로거, 장가장',
 '단간우',
 '제이미아 트래블홈',
 '인빵 블로그 : )',
 'GATEVISION 다이슨 로라스타 이메텍 비쎌 노바이러스',
 '깡이콩이 애미, 리뷰어 박씨언니',
 '리보니의 IT 리뷰 생활',
 '이야~ 기분좋다',
 '렐리아의 끄적끄적',
 '멜오',
 'JiJi',
 '로라스타 스팀다리미 공식 블로그',
 'bloghayoung',
 '게으름뱅이그녀의 패션,뷰티',
 '열심히 달리자']

In [28]:
naver_blog_crawl.search_blogs_by_API(keyword = "스매그냉장고")
blog_infos = naver_blog_crawl.html_dict["rss"]["channel"]["item"]
blog_info = naver_blog_crawl.html_dict["rss"]["channel"]["item"][0]
blog_info

OrderedDict([('title', '<b>스메그</b> 빌트인 오븐 <b>냉장고</b> 청담 플래그십 매장에서 구경하다!'),
             ('link',
              'https://blog.naver.com/cha_23?Redirect=Log&logNo=222611709924'),
             ('description',
              '오늘은 스메그 빌트인 오븐 냉장고 구경하러 스메그 청담 플래그십 다녀온 이야기해보려고 해요~ 저희... 이 제품은 돌체앤가바나 콜라보레이션 프로젝트로 완성된 ‘Sicilly is my Love’ 컬렉션 <b>스메그 냉장고</b>인데...'),
             ('bloggername', '유튜버 차수리미 V-Blog'),
             ('bloggerlink', 'https://blog.naver.com/cha_23'),
             ('postdate', '20220103')])

In [141]:
url = blog_info["link"]
url_rq = requests.get(url, verify=False)

soup = BeautifulSoup(url_rq.text, "html.parser")

blog_url = f"https://blog.naver.com/{soup.select_one('iframe')['src']}"
blog_url

'https://blog.naver.com//PostView.naver?blogId=hiphoparr&logNo=222655347970&from=search&redirect=Log&widgetTypeCall=true&directAccess=false'

In [11]:
headers = {
        "User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36"
        }

In [143]:
blog_link = blog_info["link"]
blogger_link = blog_info["bloggerlink"]

logNo = re.search(r"logNo=([\d]+)", url).group(1)
bloggerId = blogger_link.split("/")[-1]

blog_url = f"https://blog.naver.com/PostView.naver?blogId={bloggerId}&logNo={logNo}&from=search&redirect=Log&widgetTypeCall=true&directAccess=false"

In [ ]:
blog_soup

In [313]:
if blog_soup.find("div", attrs={"id" : "postViewArea"}):
    blog_contents_dict = parse_smarteditor_2(blog_soup, blog_info)
    print(1)
elif blog_soup.find("div", attrs={"class" : "se-main-container"}):
    blog_contents_dict = parse_smarteditor_one(blog_soup, blog_info)
    print(2)
elif blog_soup.find("div", attrs = {"class":"se_component_wrap sect_dsc __se_component_area"}):
    print(3)
    blog_contents_dict = parse_smarteditor_new(blog_soup, blog_info)
blog_contents_dict

1


defaultdict(list,
            {'images': ['[temp_필 조아의 느낌 있는 캠핑_20220303_img_1.jpg]'],
             'image_urls': ['https://postfiles.pstatic.net/20160930_225/cakespirit_14752427235400OlOi_JPEG/1475241405335.jpg?type=w1'],
             'contents': '[temp_필 조아의 느낌 있는 캠핑_20220303_img_1.jpg]\n\n스메그 냉장고 만들기에 대한 검색으로 제 블로그를 찾는 사람들이 요즘 좀 많은것 같아서 주방놀이 부분을 좀 상세하게 올려봅니다.\n\n방 전체 컷입니다. \n끝에 있는 서랍장은 제 잡동산이가 들어있는데\n조만간 과일을 프린트해서 넣어주려고해요.\n마치 과일과 빵창고처럼요.요리에 열올리고 계신 따님;;전체 컷 입니다. \n주방놀이는 이미 많이들 알고 계실 이케아 제품이고요~\n\n\n\n냉장고는 기존에 있던 3단 서랍장을 활용\n폼보드 5T를 두겹으로 붙이고 그위에 \n흰색 시트지를 다시 붙여 문을\n튼튼하게 만든뒤\n경첩을 활용해 고정시켰습니다.\n원목문은 일단 비싸고..\n무거워서 다칠까 걱정도 되고\n시간과 노력이 더 들고;;;\n\n스메그글씨는\n차량용 이니셜 이라고 검색하면 인터넷에서 구매하실수도 있고\n배송비가 아까우시면 이마트에 있어요.\n\n문은 롤 자석을 양면으로 붙혀주면 잘붙고 잘떨어집니다. \n\n문고리는 손잡이 닷컴 이에요.\n참고하세요.\n\n지저분한 장난감도가리고~\n제 속도 편해지고 ㅎㅎ\n\n\n\n사랑하는 요리도구 핑크핑크이건 모던하우스에서 구매했는데\n구매팁을 드리면 장난감코너말고\n키즈 요리도구를 샀다는 점\n나중에 요리할때 정말 사용해도 되고\n맘에드네요.\n\n\n아직 키가 작은 따님께선 발판을 딛고 요리를하십니다..\n\n\n그리고 간판~\n매장간판도 폼보드를 활용해서 만들었어요.폼보드로 기본틀을 잡고\n인테

In [294]:
blog_contents_dict

defaultdict(list, {'images': [], 'image_urls': [], 'contents': ''})

In [1]:
tag = {


 "taglist":[
 
   
   {"msg":"","logno":"222658923116", "tagName":"LG%EC%8A%A4%ED%83%A0%EB%B0%94%EC%9D%B4%EB%AF%B8%2C%EC%97%98%EC%A7%80%EC%8A%A4%ED%83%A0%EB%B0%94%EC%9D%B4%EB%AF%B8%2C%EC%8A%A4%ED%83%A0%EB%B0%94%EC%9D%B4%EB%AF%B8%2C%EC%8A%A4%ED%83%A0%EB%B0%94%EC%9D%B4%EB%AF%B8%ED%9B%84%EA%B8%B0%2C%EC%8A%A4%ED%83%A0%EB%B0%94%EC%9D%B4%EB%AF%B8%EC%86%94%EC%A7%81%ED%9B%84%EA%B8%B0%2Clg%EC%8A%A4%ED%83%A0%EB%B0%94%EC%9D%B4%EB%AF%B8%ED%9B%84%EA%B8%B0%2Clg%EC%8A%A4%ED%83%A0%EB%B0%94%EC%9D%B4%EB%AF%B8%EC%86%94%EC%A7%81%ED%9B%84%EA%B8%B0%2Clg%EC%8A%A4%ED%83%A0%EB%B0%94%EC%9D%B4%EB%AF%B8%EB%82%B4%EB%8F%88%EB%82%B4%EC%82%B0%ED%9B%84%EA%B8%B0%2C%EC%8A%A4%ED%83%A0%EB%B0%94%EC%9D%B4%EB%AF%B8%EC%95%84%EC%9D%B4%ED%8F%B0%EB%AF%B8%EB%9F%AC%EB%A7%81",  "encTagName":"LG%BD%BA%C5%C4%B9%D9%C0%CC%B9%CC%2C%BF%A4%C1%F6%BD%BA%C5%C4%B9%D9%C0%CC%B9%CC%2C%BD%BA%C5%C4%B9%D9%C0%CC%B9%CC%2C%BD%BA%C5%C4%B9%D9%C0%CC%B9%CC%C8%C4%B1%E2%2C%BD%BA%C5%C4%B9%D9%C0%CC%B9%CC%BC%D6%C1%F7%C8%C4%B1%E2%2Clg%BD%BA%C5%C4%B9%D9%C0%CC%B9%CC%C8%C4%B1%E2%2Clg%BD%BA%C5%C4%B9%D9%C0%CC%B9%CC%BC%D6%C1%F7%C8%C4%B1%E2%2Clg%BD%BA%C5%C4%B9%D9%C0%CC%B9%CC%B3%BB%B5%B7%B3%BB%BB%EA%C8%C4%B1%E2%2C%BD%BA%C5%C4%B9%D9%C0%CC%B9%CC%BE%C6%C0%CC%C6%F9%B9%CC%B7%AF%B8%B5"}
 
 ]
 }

In [14]:
import re

In [33]:
rq = requests.get(blog_url)
rq.json()

{'taglist': [{'msg': '',
   'logno': '222658923116',
   'tagName': 'LG%EC%8A%A4%ED%83%A0%EB%B0%94%EC%9D%B4%EB%AF%B8%2C%EC%97%98%EC%A7%80%EC%8A%A4%ED%83%A0%EB%B0%94%EC%9D%B4%EB%AF%B8%2C%EC%8A%A4%ED%83%A0%EB%B0%94%EC%9D%B4%EB%AF%B8%2C%EC%8A%A4%ED%83%A0%EB%B0%94%EC%9D%B4%EB%AF%B8%ED%9B%84%EA%B8%B0%2C%EC%8A%A4%ED%83%A0%EB%B0%94%EC%9D%B4%EB%AF%B8%EC%86%94%EC%A7%81%ED%9B%84%EA%B8%B0%2Clg%EC%8A%A4%ED%83%A0%EB%B0%94%EC%9D%B4%EB%AF%B8%ED%9B%84%EA%B8%B0%2Clg%EC%8A%A4%ED%83%A0%EB%B0%94%EC%9D%B4%EB%AF%B8%EC%86%94%EC%A7%81%ED%9B%84%EA%B8%B0%2Clg%EC%8A%A4%ED%83%A0%EB%B0%94%EC%9D%B4%EB%AF%B8%EB%82%B4%EB%8F%88%EB%82%B4%EC%82%B0%ED%9B%84%EA%B8%B0%2C%EC%8A%A4%ED%83%A0%EB%B0%94%EC%9D%B4%EB%AF%B8%EC%95%84%EC%9D%B4%ED%8F%B0%EB%AF%B8%EB%9F%AC%EB%A7%81',
   'encTagName': 'LG%BD%BA%C5%C4%B9%D9%C0%CC%B9%CC%2C%BF%A4%C1%F6%BD%BA%C5%C4%B9%D9%C0%CC%B9%CC%2C%BD%BA%C5%C4%B9%D9%C0%CC%B9%CC%2C%BD%BA%C5%C4%B9%D9%C0%CC%B9%CC%C8%C4%B1%E2%2C%BD%BA%C5%C4%B9%D9%C0%CC%B9%CC%BC%D6%C1%F7%C8%C4%B1%E2%2Clg%BD%BA%C5%C4%B9%D9%C0%C

In [52]:
rq_json

{'taglist': []}

In [55]:
blog_url = "https://blog.naver.com/PostView.naver?blogId=cha_23&logNo=222611709924&from=search&redirect=Log&widgetTypeCall=true&directAccess=false"
blogId = re.search(r"blogId=([a-zA-Z0-9_-]+)", blog_url).group(1)
logNoList = re.search(r"logNo=([\d]+)", blog_url).group(1)


params_dict = {"blogId" : blogId,
                "logNoList" : logNoList,
                "logType" : "mylog"}
base_url = "https://blog.naver.com/BlogTagListInfo.naver"

rq = requests.get(base_url, params_dict)
rq_json = rq.json()
tags = parse.unquote(rq_json["taglist"][0]["tagName"])
print(blogId)
print(logNoList)
print(tags)

cha_23
222611709924
cha_23
222611709924
스메그청담플래그십,스메그,청담플래그십,빌트인냉장고,인테리어,SMEG,신혼가전,빌트인오븐,인덕션,스메그빌트인오븐,스메그냉장고,스메그매장


In [24]:
re.search(r"blogId=([a-zA-Z0-9]+)", blog_url).group(1)

'2hailin'

In [12]:
from urllib import parse
parse.unquote(tag["taglist"][0]["tagName"])

'LG스탠바이미,엘지스탠바이미,스탠바이미,스탠바이미후기,스탠바이미솔직후기,lg스탠바이미후기,lg스탠바이미솔직후기,lg스탠바이미내돈내산후기,스탠바이미아이폰미러링'

In [9]:
tag["taglist"][0]["encTagName"].encode()

b'LG%BD%BA%C5%C4%B9%D9%C0%CC%B9%CC%2C%BF%A4%C1%F6%BD%BA%C5%C4%B9%D9%C0%CC%B9%CC%2C%BD%BA%C5%C4%B9%D9%C0%CC%B9%CC%2C%BD%BA%C5%C4%B9%D9%C0%CC%B9%CC%C8%C4%B1%E2%2C%BD%BA%C5%C4%B9%D9%C0%CC%B9%CC%BC%D6%C1%F7%C8%C4%B1%E2%2Clg%BD%BA%C5%C4%B9%D9%C0%CC%B9%CC%C8%C4%B1%E2%2Clg%BD%BA%C5%C4%B9%D9%C0%CC%B9%CC%BC%D6%C1%F7%C8%C4%B1%E2%2Clg%BD%BA%C5%C4%B9%D9%C0%CC%B9%CC%B3%BB%B5%B7%B3%BB%BB%EA%C8%C4%B1%E2%2C%BD%BA%C5%C4%B9%D9%C0%CC%B9%CC%BE%C6%C0%CC%C6%F9%B9%CC%B7%AF%B8%B5'

In [2]:
from datetime import datetime
import os
import inspect
import functools

def check_execution_time(func):
    """
        시간을 측정하는 Decorator 함수

        Args: 
            func: 대상 Function

        Returns:
            wrapper function
            
        Exception: 
    """
    @ functools.wraps(func)
    def wrapper_check_execution_time(*args, **kwargs):
        start_time = datetime.now()  

        result = func(*args, **kwargs)
        
        end_time = datetime.now()
        print(f"{func.__name__} execution time: {end_time - start_time}")
        return result
    return wrapper_check_execution_time

In [8]:
class NaverBlogCrawl:
    
    client_id = "VyJ4vDh18O7CVkYDCROr"
    client_secret = "pOn2vw75sv"
    TIME_VERBOSE = True
    def __init__(self):
        pass


    @check_execution_time
    def search_blogs_by_API(self, keyword:str) -> dict:
        blog_infos = defaultdict(list)
        self.keyword = keyword
        # API를 통한 검색
        encText = urllib.parse.quote(self.keyword)
        url = f"https://openapi.naver.com/v1/search/blog.xml?query={encText}&display=100"

        headers = {"X-Naver-Client-Id" : self.client_id,
                   "X-Naver-Client-Secret" : self.client_secret}
        rq = requests.get(url, headers=headers, verify=False)
        html = rq.text
        html_dict = xmltodict.parse(html)
        self.html_dict = html_dict

        if html_dict["rss"]["channel"]["total"] == "0":
            blog_infos = None
        else:
            blog_infos = html_dict["rss"]["channel"]["item"]

        
        return blog_infos


    @check_execution_time
    def create_blog_request_params(self, blog_info:dict) -> str:
        # 실제 Url 찾기
        blog_link = blog_info["link"]
        blogger_link = blog_info["bloggerlink"]

        logNo = re.search(r"logNo=([\d]+)", blog_link).group(1)
        bloggerId = blogger_link.split("/")[-1]

        # blog_url = f"https://blog.naver.com/PostView.naver?blogId={bloggerId}&logNo={logNo}&from=search&redirect=Log&widgetTypeCall=true&directAccess=false"
        blog_request_params = {"blogId" : bloggerId,
                               "logNo" : logNo,
                               "from" : "search",
                               "redirect" : "Log",
                               "widgetTypeCall" : "true",
                               "directAccess" :"false"}
        return blog_request_params


    @check_execution_time
    def save_images(self, img_urls:str, img_names:str, img_dir:str = "./image"):

        if img_dir is None:
            img_dir = f"./image/{self.keyword}"
        
        if not os.path.exists(img_dir):
            os.makedirs(img_dir)

        self.error_list = []
        for img_url, img_name in zip(img_urls, img_names):
            # image 저장        
            try:
                img_rq = requests.get(img_url, verify = False)
                with open(f"{img_dir}/{img_name}", "wb") as f:
                    f.write(img_rq.content)
            except: 
                self.error_list.append(img_name)
                print(f"img_name : {img_name}, img_url : {img_url}")
    

    def collect_tags(self, blog_soup:BeautifulSoup):
        blog_soup.find("a", attrs={"class":"item pcol2 itemTagfont _setTop"})
    
    @check_execution_time
    def parse_smarteditor_one(self, blog_soup:BeautifulSoup, keyword = "temp", blog_info:dict = None) -> dict:
        # https://blog.naver.com/PostView.naver?blogId=dreaminguth&logNo=222637214855&redirect=Dlog&widgetTypeCall=true&directAccess=false
        if blog_info is None:
            bloggername = "temp_user"
            postdate = "99999999"
        else:
            bloggername = blog_info["bloggername"]
            postdate = blog_info["postdate"]        

        blog_contents_dict = defaultdict(list)

        for br in blog_soup.find_all("br"):
            br.replace_with("\n")
        blog_body = blog_soup.find("div", attrs={"class":"se-main-container"})

        # 본문 내에 Image와 Text가 있는 부분만 추출
        blog_img_txt = blog_body.findAll("div", attrs={"class" : [re.compile(r"se-(module|section)-text"), re.compile(r"se-(module|section)-image")]})

        # image 파일 이름과 블로그 글을 하나의 contents로 엮어서 text 생성
        raw_blog_contents = []
        img_urls = []
        img_names = []
        img_num = 0
        if bloggername is None:
            bloggername = ""
        else:
            bloggername = re.sub(r'/|\.|\*|%|\\|:|\?|\"|\'|\<|\>|\|','_',bloggername)
        for x in blog_img_txt:
            if any([re.findall(r"se-module-image|se-section-image", y) for y in x.attrs["class"]]):
                
                try: 
                    if "data-lazy-src" in x.select_one("img").attrs.keys():
                        img_url = x.select_one("img")["data-lazy-src"]
                    else:
                        img_url = x.select_one("img")["src"]
                except:
                    continue
                
                if img_url in img_urls:
                    continue
                else:
                    img_urls.append(img_url)
                
                img_num += 1
                img_name = f"[{keyword}_{bloggername}_{postdate}_img_{img_num}.jpg]"
                img_names.append(img_name)
                raw_blog_contents.append(img_name)
            elif "se-module-text" in x.attrs["class"]:
                dom = etree.HTML(str(x))
                text_list = dom.xpath(".//p/span//text()")
                for text in text_list:
                    if text is not None:
                        raw_blog_contents.append(text)
                

        blog_contents = "\n".join(raw_blog_contents)
        
        blog_contents_dict["contents"] = blog_contents
        blog_contents_dict["images"] = img_names
        blog_contents_dict["image_urls"] = img_urls

        return blog_contents_dict

    @check_execution_time
    def parse_smarteditor_new(self, blog_soup:BeautifulSoup, keyword = "temp", blog_info:dict = None):
        if blog_info is None:
            bloggername = "temp_user"
            postdate = "99999999"
        else:
            bloggername = blog_info["bloggername"]
            postdate = blog_info["postdate"]
        blog_contents_dict = defaultdict(list)

        # 본문 내에 Image와 Text가 있는 부분만 추출
        for br in blog_soup.find_all("br"):
            br.replace_with("\n")
        blog_body = blog_soup.find("div", attrs = {"class":"se_component_wrap sect_dsc __se_component_area"})
        blog_img_txt = blog_body.findAll("div", attrs={"class" : ["se_component se_paragraph default", "se_component se_image default"]})

        # image 파일 이름과 블로그 글을 하나의 contents로 엮어서 text 생성
        raw_blog_contents = []
        img_urls = []
        img_names = []
        img_num = 0
        if bloggername is None:
            bloggername = ""
        else:
            bloggername = re.sub(r'/|\.|\*|%|\\|:|\?|\"|\'|\<|\>|\|','_',bloggername)
        for x in blog_img_txt:
            if "se_image" in x.attrs["class"]:
                img_num += 1
                if "data-lazy-src" in x.select_one("img").attrs.keys():
                    img_urls.append(x.select_one("img")["data-lazy-src"])
                else:
                    img_urls.append(x.select_one("img")["src"])
                
                img_name = f"[{keyword}_{bloggername}_{postdate}_img_{img_num}.jpg]"
                img_names.append(img_name)
                raw_blog_contents.append(img_name)
            elif "se_paragraph" in x.attrs["class"]:
                text = x.get_text()
                raw_blog_contents.append(text)

        blog_contents = "\n".join(raw_blog_contents)
        
        blog_contents_dict["contents"] = blog_contents
        blog_contents_dict["images"] = img_names
        blog_contents_dict["image_urls"] = img_urls

        return blog_contents_dict


    @check_execution_time
    def parse_smarteditor_2(self, blog_soup:BeautifulSoup, keyword = "temp", blog_info:dict = None):
        if blog_info is None:
            bloggername = "temp_user"
            postdate = "99999999"
        else:
            bloggername = blog_info["bloggername"]
            postdate = blog_info["postdate"]
        blog_contents_dict = defaultdict(list)

        # 본문 내에 Image와 Text가 있는 부분만 추출
        for br in blog_soup.find_all("br"):
            br.replace_with("\n")        
        blog_body = blog_soup.find("div", attrs = {"id":"postViewArea"})
        blog_img_txt = blog_body.findAll("p")
        if len(blog_img_txt) == 0:
            blog_img_txt = blog_body.findAll("div", attrs = {"id":re.compile(r"post-view")})

        # image 파일 이름과 블로그 글을 하나의 contents로 엮어서 text 생성
        raw_blog_contents = []
        img_urls = []
        img_names = []
        img_num = 0
        if bloggername is None:
            bloggername = ""
        else:
            bloggername = re.sub(r'/|\.|\*|%|\\|:|\?|\"|\'|\<|\>|\|','_',bloggername)
        for x in blog_img_txt:
            if x.find("img", attrs={"class": "_photoImage"}) is not None:
                img_num += 1
                if "data-lazy-src" in x.find("img", attrs={"class": "_photoImage"}).attrs.keys():
                    img_urls.append(x.find("img", attrs={"class": "_photoImage"})["data-lazy-src"])
                else:
                    img_urls.append(x.find("img", attrs={"class": "_photoImage"})["src"])
                
                img_name = f"[{keyword}_{bloggername}_{postdate}_img_{img_num}.jpg]"
                img_names.append(img_name)
                raw_blog_contents.append(img_name)

                text = x.get_text()
                raw_blog_contents.append(text)
            else:
                text = x.get_text()
                raw_blog_contents.append(text)

        blog_contents = "\n".join(raw_blog_contents)

        blog_contents_dict["images"] = img_names
        blog_contents_dict["image_urls"] = img_urls
        blog_contents_dict["contents"] = blog_contents

        return blog_contents_dict


    @check_execution_time
    def get_blog_soup(self, blog_request_params):
        # Blog URL에서 작성자, 날짜, 본문 추출
        BLOG_BASE_URL = "https://blog.naver.com/PostView.naver"
        blog_rq = requests.get(BLOG_BASE_URL, params = blog_request_params, verify=False)
        blog_soup = BeautifulSoup(blog_rq.text, "html.parser")

        return blog_soup, blog_rq.url
        

    @check_execution_time
    def extract_contents(self, blog_soup, keyword, blog_info:dict):
        
        # # Blog URL에서 작성자, 날짜, 본문 추출
        # BLOG_BASE_URL = "https://blog.naver.com/PostView.naver"
        # blog_rq = requests.get(BLOG_BASE_URL, params = blog_request_params, verify=False)
        # blog_soup = BeautifulSoup(blog_rq.text, "html.parser")

        # self.blog_soup = blog_soup
        # # 작성자 ID
        # writer_name = blog_soup.find("span", attrs = {"class" : "nick"}).get_text().replace(" ", "_")

        # # 날짜
        # post_date = blog_soup.find("span", attrs = {"class" : "se_publishDate pcol2"}).get_text().replace(" ", "")
        # post_date = re.sub("\\.|:|\s", "_", post_date)

        # 블로그 본문
        if blog_soup.find("div", attrs={"id" : "postViewArea"}):
            blog_type = "parse_smarteditor_2"
            blog_contents_dict = self.parse_smarteditor_2(blog_soup, keyword, blog_info)
        elif blog_soup.find("div", attrs={"class" : "se-main-container"}):
            blog_type = "parse_smarteditor_one"
            blog_contents_dict = self.parse_smarteditor_one(blog_soup, keyword, blog_info)
        elif blog_soup.find("div", attrs = {"class":"se_component_wrap sect_dsc __se_component_area"}):
            blog_type = "parse_smarteditor_new"
            blog_contents_dict = self.parse_smarteditor_new(blog_soup, keyword, blog_info)
        else:
            blog_type = "Else"
        
        blog_contents_dict["contents"] = re.sub(r"[\x00-\x08\x0E-\x1F\x7F]+"," ", blog_contents_dict["contents"]["contents"])


        return blog_contents_dict, blog_type

    def merge_dict(self, org_dict:dict, new_dict:dict, type:str = "full"):
        merged_dict = deepcopy(org_dict)
        if type == "full":
            keys = pd.unique(list(org_dict.keys()) + list(new_dict.keys()))
        elif type == "left":
            keys = list(org_dict.keys())
        elif type == "right":
            keys = list(new_dict.keys())


        for key in keys:
            if isinstance(new_dict[key], list):
                merged_dict[key].extend(new_dict[key])
            else:
                merged_dict[key].append(new_dict[key])
        
        return merged_dict

    @check_execution_time
    def collect_blog(self, keywords:Union[str,list[str]]):
        self.result_dict = defaultdict(list)
        if not isinstance(keywords, list):
            self.keywords = [keywords]
        else:
            self.keywords = keywords
        
        for i, keyword in enumerate(self.keywords):
            # if i < 1460:
            #     continue
            keyword_result_dict = defaultdict(list)
            
            self.blog_infos = self.search_blogs_by_API(keyword)
            if self.blog_infos is None:
                continue
            if isinstance(self.blog_infos, dict):
                self.blog_infos = [self.blog_infos]
            self.blog_infos = [blog_info for blog_info in self.blog_infos if "https://blog.naver.com" in blog_info["link"]] 
            
            print(f"{i}, keyword : {keyword}, # of blogs : {len(self.blog_infos)}")
            
            img_dir = os.path.join("./image", keyword)

            for _, blog_info in enumerate(self.blog_infos):
                try:
                    self.blog_info = blog_info
                    self.blog_request_params = self.create_blog_request_params(blog_info)
                    blog_soup, blog_url = self.get_blog_soup(self.blog_request_params)
                    blog_contents_dict, blog_type = self.extract_contents(blog_soup, keyword, blog_info)
                    # self.save_images(blog_contents_dict["image_urls"], blog_contents_dict["images"], img_dir = img_dir)
                except Exception as e:
                    print(e)
                    continue
                # start_time = datetime.now()
                keyword_result_dict["keyword"].append(keyword)
                keyword_result_dict["blog_type"].append(blog_type)
                keyword_result_dict["title"].append(blog_info["title"])
                keyword_result_dict["writer_name"].append(blog_info["bloggername"])
                keyword_result_dict["post_date"].append(blog_info["postdate"])
                keyword_result_dict["blog_url"].append(blog_url)
                keyword_result_dict["contents"].append(blog_contents_dict["contents"])
                keyword_result_dict["images"].append(blog_contents_dict["images"])
                keyword_result_dict["image_urls"].append(blog_contents_dict["image_urls"])
                # end_time = datetime.now()
                # print(f"insert dict Execution Time : {end_time - start_time}" )

            self.result_dict = self.merge_dict(org_dict = self.result_dict, new_dict = keyword_result_dict) 

            file_keyword = keyword.replace("/","_")
            with open(f"./blog_dict/{file_keyword}_blog_dict.pkl", "wb") as f:
                pickle.dump(keyword_result_dict,f)

            if (i+1) % 100 == 0:
                pd.DataFrame(self.result_dict).to_excel(f"Naver_Blog_{i+1-100}_{i+1}.xlsx", index = False, encoding = "CP949")
                # self.result_dict = defaultdict(list)
                # pd.DataFrame(self.result_dict).to_csv(f"Naver_Blog_{i}.csvw", index = False, encoding = "CP949")
        return self.result_dict


In [9]:
naver_blog_crawl = NaverBlogCrawl()
result = naver_blog_crawl.collect_blog(keywords = "LG스탠바이미")

search_blogs_by_API execution time: 0:00:00.256566
0, keyword : LG스탠바이미, # of blogs : 97
create_blog_request_params execution time: 0:00:00
get_blog_soup execution time: 0:00:00.295015
parse_smarteditor_one execution time: 0:00:00.014002
string indices must be integers
create_blog_request_params execution time: 0:00:00
get_blog_soup execution time: 0:00:00.235630
parse_smarteditor_one execution time: 0:00:00.009000
string indices must be integers
create_blog_request_params execution time: 0:00:00
get_blog_soup execution time: 0:00:00.183295
parse_smarteditor_one execution time: 0:00:00.008000
string indices must be integers
create_blog_request_params execution time: 0:00:00
get_blog_soup execution time: 0:00:00.212303
parse_smarteditor_one execution time: 0:00:00.009004
string indices must be integers
create_blog_request_params execution time: 0:00:00
get_blog_soup execution time: 0:00:00.253766
parse_smarteditor_one execution time: 0:00:00.013999
string indices must be integers
create

In [228]:
naver_blog_crawl = NaverBlogCrawl()
result = naver_blog_crawl.collect_blog(keywords = SEARCH_KEYWORDS)


1460, keyword : PT-RZ770, # of blogs : 9
1461, keyword : PT-RX110, # of blogs : 10
1462, keyword : GX511, # of blogs : 100
1463, keyword : MS504, # of blogs : 98
1464, keyword : MX511, # of blogs : 22
1465, keyword : MW843UST, # of blogs : 38
1466, keyword : BS274, # of blogs : 16
1467, keyword : HW300G, # of blogs : 18
1468, keyword : W710ST, # of blogs : 99
1469, keyword : PT-DZ10KE, # of blogs : 100
1470, keyword : PW9520, # of blogs : 1
1471, keyword : PT-CW330, # of blogs : 13
1472, keyword : PT-RW620, # of blogs : 3
1473, keyword : MS500, # of blogs : 96
1474, keyword : PU9730, # of blogs : 8
1475, keyword : MX823ST, # of blogs : 4
1476, keyword : HW300TN, # of blogs : 93
1477, keyword : MX822ST, # of blogs : 100
1478, keyword : MW516, # of blogs : 16
1479, keyword : TH681, # of blogs : 3
1480, keyword : Joybee GP2, # of blogs : 26
1482, keyword : MW883UST, # of blogs : 1
1483, keyword : PT-DX610E, # of blogs : 100
1484, keyword : PT-RW730, # of blogs : 6
1486, keyword : PT-DZ870

In [181]:
naver_blog_crawl.result_dict.keys()

dict_keys(['keyword', 'blog_type', 'title', 'writer_name', 'post_date', 'blog_url', 'contents', 'images', 'image_urls', 'content'])

In [220]:
temp = pd.DataFrame(naver_blog_crawl.result_dict).iloc[668]

In [221]:
temp["contents"] = re.sub(r"[\x00-\x08\x0E-\x1F\x7F]+"," ",temp["contents"])

In [223]:
temp.to_excel("temp.xlsx")

In [216]:
print(pd.DataFrame(naver_blog_crawl.result_dict).iloc[668]["contents"])







   지금 상품 구매후 리뷰 참여이벤트 진행중입니다. 많은 참여부탁드립니다 .    . . . . . . . . . . . . . . .   















 LG기네 다나와 빔 pH550. 












맺음말부터 미니빔 말하자면. 아이폰 




















[PH550_ATICA아티카_20220115_img_1.jpg]






 강세 받았던 지난날에 이 다나와 빔으로 흰 벽을 가득가득 보충하다 실은 가탄을 금치 못했기 때문에.. 실은 첨예하다.


























 바라며. 상업광고 휴대용 아니고 실은 내 돈대산으로 감명 준 받아야 하는 여러분도 나와 같은 감명 준을 느끼시길.


























 ㅜ 작은 전 영국관. 옛날에 작성했던 물표 위험성 이래로 이번 편은 맹세한 후반기와 작용에 대한 담론! 






[PH550_ATICA아티카_20220115_img_2.jpg]






방 한 칸 + 흰 두멧골 + 빈 벽 있는 명이라면 제발 사주세요. 장단과 진솔하다가 후반기 남겨 본다


























참작으로, 미니빔 이 글은 나와 같이












빔프로젝터를 고장 싶은데, 다나와 아이폰, 미니빔 fhd 맥북, 아이패드 사용자다 하시는 분들을 다나와 위한 글이다.


























 첫날 이것저것 무선 가설 하고 싶네 빔을 켜서 아이폰 비디오테이프리코더를 송출해봤는데 아이폰 실은 많이 감명 캠핑 미니빔 준했다. 












크롬캐스트 결합 후 송출되고 있는 기해 비디오 테이프리코더


























왓챠플레이 볼을 결합하여 송출되는 전 영국 비디오 테이프리코더






[PH550_ATICA아티카_20220115_img_3.jpg]




















일단, 조영부터 보여드리겠다.


























In [201]:
pd.DataFrame(naver_blog_crawl.result_dict).head(669).to_excel("temp.xlsx")

IllegalCharacterError: 

In [160]:
naver_blog_crawl.html_dict

OrderedDict([('rss',
              OrderedDict([('@version', '2.0'),
                           ('channel',
                            OrderedDict([('title',
                                          "Naver Open API - blog ::'Crystal UHD KU65UA7070FXKR'"),
                                         ('link', 'https://search.naver.com'),
                                         ('description',
                                          'Naver Search Result'),
                                         ('lastBuildDate',
                                          'Thu, 10 Mar 2022 03:54:15 +0900'),
                                         ('total', '0'),
                                         ('start', '1'),
                                         ('display', '0')]))]))])

In [87]:
temp = pd.DataFrame(result)
temp.to_excel("블로그 수집.xlsx", index =False)

In [266]:
temp.contents[21]

'하이마트 다이슨 선풍기 에어플라이어 날개없는 AM07-WS\n영국 혁신기업 다이슨의 특허받은\n에어멀트플라이어 기술을 활용한\n또 한번의 혁신 다이슨 am\n공기의 흐름이 단절없이 부드럽고\n시원한 자연풍의 바람을 만들어내는\n다이슨 만의 에어멀티플라이어 기술\nam02보다 60% 더 조용해진 am07\n설정된 시간에 자동으로\n전원이 종료되도록 타이머 기능이\n추가되어 수면 중에도 안전하게\n사용이 가능합니다.\n하이마트에서 혜택가로\n구매하세요\ne-himart.co.kr/app/goods/goodsDetail?goodsNo=0000000197\n다이슨선풍기_쿠폰프레스★CouponPress_20210518_img_1.jpg\n\n\n간편하게 원거리에서 사용할 수 있도록\n리모컨이 포함되어 있으며\xa0\n제품에 붙여 보관하기 때문에 분실 염려가\n없음.\n날개가 없기 때문에 아이가 있는 집에서도\n매우 안전하며 매우 간편하게\n청소와 보관이 가능함.\n다이슨선풍기_쿠폰프레스★CouponPress_20210518_img_2.jpg\n'